In [1]:
# imports
import requests
import pandas as pd
import os
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
apikey = os.environ['FOURSQUARE_API_KEY']

In [3]:
# find bike stations within 1000 miles
def bike_stations(latitude, longitude):
    url = 'https://api.foursquare.com/v3/places/search'
    
    params = {
        'radius': '1000',
    }
    
    headers = {
        'Accept': 'application/json',
        'Authorization': os.environ['FOURSQUARE_API_KEY']
    }
    
    # Get the request
    response = requests.get(url, params=params, headers=headers)
    
    # Convert to Python object with .json()
    res = response.json()
    
    if 'results' in res:
        return len(res['results'])
    else:
        return 0

In [4]:
# read csv file
df = pd.read_csv("station_data.csv")

# Extract Latitude and Longitude columns from the DataFrame
station_id = df['station_id']
latitude = df['Latitude']
longitude = df['Longitude']

# Create a new DataFrame to store the results
results_df = pd.DataFrame()

# Set a counter to limit the number of rows
counter = 0

# Iterate over each latitude and longitude pair
for station, lat, lng in zip(station_id, latitude, longitude):
    # Call the get_venues function with the current latitude and longitude
    num_results = bike_stations(latitude, longitude)
    
    # Create a new DataFrame with the latitude, longitude, and number of stations
    new_row = pd.DataFrame({'Station ID': [station], 'Latitude': [lat], 'Longitude': [lng], 'Num of Businesses': [num_results]})
    
    # Concatenate the new row with the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    # Increment the counter
    counter += 1
    
    # Check if the counter reaches 50
    if counter == 50:
        break

# show the results
results_df.head()


,Station ID,Latitude,Longitude,Num of Businesses
0,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10
1,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,10
2,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,10
3,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,10
4,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,10


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
poi_location_ids = []

def get_poi_location_ids(station_id):
    url = "https://api.foursquare.com/v3/places/search"
    
    params = {
        "categories": "16032",
        "ll": str(coords),
        "radius": "1000",
        "limit": "5"
    }
    
    headers = {
        "Accept": "application/json",
        "Authorization": os.environ['FOURSQUARE_API_KEY']
    }
    
    # Get the request
    response = requests.get(url, params=params, headers=headers)
   
    # Convert to Python object with .json()
    res = response.json()
 
    # Create a dictionary for each result
    for i in range(len(res['results'])):
        location = {
            'station_id': station_id,
            'fsq_id': res['results'][i]['fsq_id'],
            'fsq_lat': res['results'][i]['geocodes']['main']['latitude'],
            'fsq_long': res['results'][i]['geocodes']['main']['longitude']
        }
        poi_location_ids.append(location)

In [6]:
# Data file
data_file = "station_data.csv"

# Read the data file
df = pd.read_csv(data_file)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    station_id = row['station_id']
    num_of_slots = row['Num_of_Slots']
    get_poi_location_ids(station_id)

    # Check if the number of locations exceeds 50
    if len(poi_location_ids) >= 50:
         break

# Create a list of dictionaries containing fsq_id, and station_id
location_dicts = []
for location in poi_location_ids:
    location_dict = {
            'station_id': station_id,
            'fsq_id': location['fsq_id'],
            'fsq_lat': location['fsq_lat'],
            'fsq_long': location['fsq_long']
    }
    location_dicts.append(location_dict)


In [7]:
# Convert the list of dictionaries into a DataFrame
poi_locations_df = pd.DataFrame(location_dicts)

# Remove duplicates in the 'fsq_id' column of poi_locations_df
poi_locations_df = poi_locations_df.drop_duplicates(subset='fsq_id')

# Print the updated poi_locations_df without duplicates
poi_locations_df.head()

,station_id,fsq_id,fsq_lat,fsq_long
0,6993b3dbb0758927967592ea612a2b1e,4be73b80910020a16435d514,49.264274,-123.108018
1,6993b3dbb0758927967592ea612a2b1e,4aada430f964a5201f6120e3,49.266831,-123.124538
2,6993b3dbb0758927967592ea612a2b1e,4e700331d1649bf26fdfe4eb,49.256478,-123.105651
3,6993b3dbb0758927967592ea612a2b1e,4c0beac07e3fc928d200f682,49.270777,-123.109869
4,6993b3dbb0758927967592ea612a2b1e,4d486a1bf046a1cd8f7611f6,49.271004,-123.110781


In [8]:
# create a list of point of interest information
poi_info_list = []

# Set a counter to keep track of rows processed
counter = 0

# get fsq_id, name, rating, location
for f_id in poi_locations_df['fsq_id']:
    
    if counter >= 50:
        break  # Stop processing if the limit is reached

    url = "https://api.foursquare.com/v3/places/" + f_id

    params = {
        "fields": "fsq_id,name,rating,popularity,location"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": os.environ['FOURSQUARE_API_KEY']
    }

    response = requests.request("GET", url, params=params, headers=headers)
    res = response.json()
    poi_info_list.append(res)

    counter += 1

Put your parsed results into a DataFrame

In [9]:
# create dataframe
parks_df = pd.json_normalize(poi_info_list)

# show df
parks_df.head()

,fsq_id,name,popularity,rating,location.address,location.country,location.formatted_address,location.locality,location.postcode,location.region,location.cross_street,location.address_extended
0,4be73b80910020a16435d514,Jonathan Rogers Park,0.968476,8.2,110 7th Ave W,CA,"110 7th Ave W, Vancouver BC V5Y 1L7",Vancouver,V5Y 1L7,BC,NaN,NaN
1,4aada430f964a5201f6120e3,Charleson Park,0.992559,8.9,999 Charleson St,CA,"999 Charleson St, Vancouver BC",Vancouver,NaN,BC,NaN,NaN
2,4e700331d1649bf26fdfe4eb,Mt. Pleasant Park,0.987758,8.4,32 16th Ave W,CA,"32 16th Ave W, Vancouver BC V5Y 1Y6",Vancouver,V5Y 1Y6,BC,,NaN
3,4c0beac07e3fc928d200f682,Hinge Park,0.985902,7.4,Columbia St,CA,"Columbia St (at Walter Harkwick Ave), Vancouve...",Vancouver,NaN,BC,at Walter Harkwick Ave,NaN
4,4d486a1bf046a1cd8f7611f6,Olympic Village Dog Run,0.875586,6.8,NaN,CA,Vancouver BC,Vancouver,NaN,BC,,NaN


In [10]:
# merged poi_locations_df and parks_df
merged_fsq_df = pd.merge(poi_locations_df, parks_df, on='fsq_id', how='inner')

# view merged file
merged_fsq_df.head()

,station_id,fsq_id,fsq_lat,fsq_long,name,popularity,rating,location.address,location.country,location.formatted_address,location.locality,location.postcode,location.region,location.cross_street,location.address_extended
0,6993b3dbb0758927967592ea612a2b1e,4be73b80910020a16435d514,49.264274,-123.108018,Jonathan Rogers Park,0.968476,8.2,110 7th Ave W,CA,"110 7th Ave W, Vancouver BC V5Y 1L7",Vancouver,V5Y 1L7,BC,NaN,NaN
1,6993b3dbb0758927967592ea612a2b1e,4aada430f964a5201f6120e3,49.266831,-123.124538,Charleson Park,0.992559,8.9,999 Charleson St,CA,"999 Charleson St, Vancouver BC",Vancouver,NaN,BC,NaN,NaN
2,6993b3dbb0758927967592ea612a2b1e,4e700331d1649bf26fdfe4eb,49.256478,-123.105651,Mt. Pleasant Park,0.987758,8.4,32 16th Ave W,CA,"32 16th Ave W, Vancouver BC V5Y 1Y6",Vancouver,V5Y 1Y6,BC,,NaN
3,6993b3dbb0758927967592ea612a2b1e,4c0beac07e3fc928d200f682,49.270777,-123.109869,Hinge Park,0.985902,7.4,Columbia St,CA,"Columbia St (at Walter Harkwick Ave), Vancouve...",Vancouver,NaN,BC,at Walter Harkwick Ave,NaN
4,6993b3dbb0758927967592ea612a2b1e,4d486a1bf046a1cd8f7611f6,49.271004,-123.110781,Olympic Village Dog Run,0.875586,6.8,NaN,CA,Vancouver BC,Vancouver,NaN,BC,,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
yelp_apikey = os.environ["yelp_api"]

In [12]:
def yelp_bike_stations(lat_list, long_list):
    yelp_list = []

    for lat, long in zip(lat_list, long_list):
        url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(long) + "&radius=1000&categories=bars&sort_by=best_match&limit=50"

        headers = {
            "accept": "application/json",
            "Authorization": os.environ["yelp_api"]
        }

        response = requests.get(url, headers=headers)
        data = response.json()

        if 'businesses' in data:
            yelp_list.extend(data['businesses'])

        # Check if the limit of 25 iterations is reached
        if len(yelp_list) >= 25:
            break

    return len(yelp_list)

In [13]:
# Read csv file
df = pd.read_csv("station_data.csv")

# Extract Latitude and Longitude columns from the DataFrame
station_id = df['station_id']
latitude = df['Latitude']
longitude = df['Longitude']

# Create a new DataFrame to store the results
results_df = pd.DataFrame()

# Set a counter to limit the number of rows
counter = 0

# Iterate over each latitude and longitude pair
for lat, lng in zip(latitude, longitude):
    # Call the get_venues function with the current latitude and longitude
    num_results = yelp_bike_stations(latitude, longitude)
    
    # Create a new DataFrame with the latitude, longitude, and number of stations
    new_row = pd.DataFrame({'Station_ID': [station_id], 'Latitude': [lat], 'Longitude': [lng], 'Num of Businesses': [num_results]})
    
    # Concatenate the new row with the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    # Increment the counter
    counter += 1
    
    # Check if the counter reaches 25
    if counter == 25:
        break

# Print the results DataFrame
results_df.head()

,Station_ID,Latitude,Longitude,Num of Businesses
0,0 7a19c49f486d7c0c02b3685d7b240448 1 ...,49.262487,-123.114397,26
1,0 7a19c49f486d7c0c02b3685d7b240448 1 ...,49.274566,-123.121817,26
2,0 7a19c49f486d7c0c02b3685d7b240448 1 ...,49.279764,-123.110154,26
3,0 7a19c49f486d7c0c02b3685d7b240448 1 ...,49.260599,-123.113504,26
4,0 7a19c49f486d7c0c02b3685d7b240448 1 ...,49.264215,-123.117772,26


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [14]:
yelp_location_ids = []

def get_yelp_location_ids(coords, station_id):
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(coords[0]) + "&longitude=" + str(coords[1]) + "&radius=1000&categories=parks&sort_by=best_match&limit=20"

    headers = {
        "accept": "application/json",
        "Authorization": os.environ["yelp_api"]
    }

    response = requests.get(url, headers=headers)

    res = response.json()

    for i in range(len(res['businesses'])):
        location = {
            'station_id': station_id,
            'id': res['businesses'][i]['id'],
            'name': res['businesses'][i]['name']
        }
        yelp_location_ids.append(location)


In [15]:
# Data file
data_file = "station_data.csv"

# Read the data file
df = pd.read_csv(data_file)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    coords = [row['Latitude'], row['Longitude']]
    station_id = row['station_id']
    get_yelp_location_ids(coords, station_id)

    # Check if the number of locations exceeds 25
    if len(yelp_location_ids) >= 25:
        break

In [16]:
location_dicts = []
counter = 0

for location in yelp_location_ids:
    location_dict = {
        'station_id': location['station_id'],
        'id': location['id'],
    }
    location_dicts.append(location_dict)
    
    counter += 1
    if counter == 25:
        break

In [17]:
# Convert the list of dictionaries into a DataFrame
yelp_locations_df = pd.DataFrame(location_dicts)

# rename id to yelp_id
yelp_locations_df = yelp_locations_df.rename(columns={'id': 'yelp_id'})

# Print the updated yelp_locations_df without duplicates
yelp_locations_df.head()

,station_id,yelp_id
0,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw
1,7a19c49f486d7c0c02b3685d7b240448,E43R3DuzaKD05c-Ngv-4Nw
2,7a19c49f486d7c0c02b3685d7b240448,-65-OqxIBL4qdTN5PA-svw
3,7a19c49f486d7c0c02b3685d7b240448,tI70Wf2bTa2eH2rURHYp5Q
4,7a19c49f486d7c0c02b3685d7b240448,yoCA7b04Q_f_HQX__EDchA


In [18]:
# create a list of point of interest information
yelp_info_list = []

# Set a counter to keep track of rows processed
counter = 0

# get yelp information
for y_id in yelp_locations_df['yelp_id']:
    
    if counter >= 25:
        break  # Stop processing if the limit is reached

    url = 'https://api.yelp.com/v3/businesses/' + str(y_id)

    headers = {
        "accept": "application/json",
        "Authorization": os.environ["yelp_api"]
    }

    response = requests.get(url, headers=headers)

    res = response.json()
    yelp_info_list.append(res)

    counter += 1

In [19]:
# create dataframe
yelp_df = pd.json_normalize(yelp_info_list)

# rename id to yelp_id
yelp_df = yelp_df.rename(columns={'id': 'yelp_id'})

# show df
yelp_df.head()

,yelp_id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude
0,oCNy5AtXUagtQQQo0vqAtw,charleson-park-vancouver-2,Charleson Park,https://s3-media1.fl.yelpcdn.com/bphoto/aC7cpG...,False,False,https://www.yelp.com/biz/charleson-park-vancou...,,,20,...,,,Vancouver,V5Z,CA,BC,"[999 Charleson Street, Vancouver, BC V5Z, Canada]",,49.266156,-123.125423
1,E43R3DuzaKD05c-Ngv-4Nw,jonathan-rogers-park-vancouver,Jonathan Rogers Park,https://s3-media1.fl.yelpcdn.com/bphoto/4P8PaC...,False,False,https://www.yelp.com/biz/jonathan-rogers-park-...,+16042578400,+1 604-257-8400,11,...,,,Vancouver,V5Y 1L7,CA,BC,"[110 7 Avenue W, Vancouver, BC V5Y 1L7, Canada]",,49.264301,-123.107964
2,-65-OqxIBL4qdTN5PA-svw,mount-pleasant-park-vancouver,Mount Pleasant Park,https://s3-media1.fl.yelpcdn.com/bphoto/zUA_WC...,False,False,https://www.yelp.com/biz/mount-pleasant-park-v...,,,4,...,,,Vancouver,V5T 2Z1,CA,BC,"[3161 Ontario Street, Vancouver, BC V5T 2Z1, C...",,49.257130,-123.106250
3,tI70Wf2bTa2eH2rURHYp5Q,major-matthews-park-vancouver,Major Matthews Park,https://s3-media3.fl.yelpcdn.com/bphoto/QH5MUd...,False,False,https://www.yelp.com/biz/major-matthews-park-v...,,,1,...,,,Vancouver,V5Y 3A9,CA,BC,"[2627 Manitoba Street, Vancouver, BC V5Y 3A9, ...",,49.261572,-123.107141
4,yoCA7b04Q_f_HQX__EDchA,heather-park-vancouver,Heather Park,https://s3-media1.fl.yelpcdn.com/bphoto/A8-rJf...,False,False,https://www.yelp.com/biz/heather-park-vancouve...,+16048737000,+1 604-873-7000,2,...,,None,Vancouver,V5Z 1W1,CA,BC,"[702 W 18 Avenue, Vancouver, BC V5Z 1W1, Canada]",,49.255065,-123.120837


In [20]:
# merge yelp_locations_df and yelp_df
merged_yelp_df = pd.merge(yelp_locations_df, yelp_df, on='yelp_id', how='inner')

# show merged df
merged_yelp_df.head()

,station_id,yelp_id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,...,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude
0,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,charleson-park-vancouver-2,Charleson Park,https://s3-media1.fl.yelpcdn.com/bphoto/aC7cpG...,False,False,https://www.yelp.com/biz/charleson-park-vancou...,,,...,,,Vancouver,V5Z,CA,BC,"[999 Charleson Street, Vancouver, BC V5Z, Canada]",,49.266156,-123.125423
1,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,charleson-park-vancouver-2,Charleson Park,https://s3-media1.fl.yelpcdn.com/bphoto/aC7cpG...,False,False,https://www.yelp.com/biz/charleson-park-vancou...,,,...,,,Vancouver,V5Z,CA,BC,"[999 Charleson Street, Vancouver, BC V5Z, Canada]",,49.266156,-123.125423
2,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,charleson-park-vancouver-2,Charleson Park,https://s3-media1.fl.yelpcdn.com/bphoto/aC7cpG...,False,False,https://www.yelp.com/biz/charleson-park-vancou...,,,...,,,Vancouver,V5Z,CA,BC,"[999 Charleson Street, Vancouver, BC V5Z, Canada]",,49.266156,-123.125423
3,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,charleson-park-vancouver-2,Charleson Park,https://s3-media1.fl.yelpcdn.com/bphoto/aC7cpG...,False,False,https://www.yelp.com/biz/charleson-park-vancou...,,,...,,,Vancouver,V5Z,CA,BC,"[999 Charleson Street, Vancouver, BC V5Z, Canada]",,49.266156,-123.125423
4,7a19c49f486d7c0c02b3685d7b240448,E43R3DuzaKD05c-Ngv-4Nw,jonathan-rogers-park-vancouver,Jonathan Rogers Park,https://s3-media1.fl.yelpcdn.com/bphoto/4P8PaC...,False,False,https://www.yelp.com/biz/jonathan-rogers-park-...,+16042578400,+1 604-257-8400,...,,,Vancouver,V5Y 1L7,CA,BC,"[110 7 Avenue W, Vancouver, BC V5Y 1L7, Canada]",,49.264301,-123.107964


Put your parsed results into a DataFrame

In [21]:
# required columns
merged_yelp_df = merged_yelp_df[['station_id','yelp_id','name','review_count', 'rating']]

# show revised df
merged_yelp_df.head()

,station_id,yelp_id,name,review_count,rating
0,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
1,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
2,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
3,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
4,7a19c49f486d7c0c02b3685d7b240448,E43R3DuzaKD05c-Ngv-4Nw,Jonathan Rogers Park,11,4.5


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp's API provided more information, however the Foursquare API was easier to work with and I was able to discover more about the data with multiple inquires. On the other hand, I faced was with the limited calls available in Yelp's API. 

Get the top 10 parks according to their rating

# Foursquare API

In [22]:
# Sort the parks DataFrame by rating in descending order
filtered_fsq_df = merged_fsq_df.sort_values('rating', ascending=False)

# Print the sorted DataFrame
filtered_fsq_df.head(10)

,station_id,fsq_id,fsq_lat,fsq_long,name,popularity,rating,location.address,location.country,location.formatted_address,location.locality,location.postcode,location.region,location.cross_street,location.address_extended
5,6993b3dbb0758927967592ea612a2b1e,4b572f20f964a520f92928e3,49.272330,-123.124359,David Lam Park,0.998757,9.3,1300 Pacific Blvd,CA,"1300 Pacific Blvd, Vancouver BC V6Z 0A9",Vancouver,V6Z 0A9,BC,NaN,NaN
6,6993b3dbb0758927967592ea612a2b1e,4aa99f79f964a5208e5420e3,49.272361,-123.129390,George Wainborn Park,0.997245,9.0,450 Beach Cres,CA,"450 Beach Cres (at Richards St.), Vancouver BC...",Vancouver,V6Z 3E5,BC,at Richards St.,NaN
15,6993b3dbb0758927967592ea612a2b1e,4c474fc476d72d7fa3c03c4d,49.289362,-123.121757,Harbour Green Park,0.973870,9.0,1187A1 W Cordova St,CA,"1187A1 W Cordova St (Bute St.), Vancouver BC V...",Vancouver,V6C 3R4,BC,Bute St.,NaN
1,6993b3dbb0758927967592ea612a2b1e,4aada430f964a5201f6120e3,49.266831,-123.124538,Charleson Park,0.992559,8.9,999 Charleson St,CA,"999 Charleson St, Vancouver BC",Vancouver,NaN,BC,NaN,NaN
7,6993b3dbb0758927967592ea612a2b1e,4aa9d826f964a5206b5520e3,49.273486,-123.114075,Cooper's Park,0.997265,8.9,1020 Marinaside Cres,CA,"1020 Marinaside Cres (Cambie Bridge), Vancouve...",Vancouver,NaN,BC,Cambie Bridge,NaN
9,6993b3dbb0758927967592ea612a2b1e,4b7c2ccaf964a5201a822fe3,49.274585,-123.102893,Creekside Park,0.998852,8.9,1455 Quebec St,CA,"1455 Quebec St (at Science World), Vancouver B...",Vancouver,V6A 3Z7,BC,at Science World,NaN
2,6993b3dbb0758927967592ea612a2b1e,4e700331d1649bf26fdfe4eb,49.256478,-123.105651,Mt. Pleasant Park,0.987758,8.4,32 16th Ave W,CA,"32 16th Ave W, Vancouver BC V5Y 1Y6",Vancouver,V5Y 1Y6,BC,,NaN
10,6993b3dbb0758927967592ea612a2b1e,4aaaf0f4f964a5202a5820e3,49.284566,-123.101795,Crab Park,0.909406,8.4,101 Waterfront Rd E,CA,"101 Waterfront Rd E (at Main St.), Vancouver B...",Vancouver,V6A 4K3,BC,at Main St.,NaN
0,6993b3dbb0758927967592ea612a2b1e,4be73b80910020a16435d514,49.264274,-123.108018,Jonathan Rogers Park,0.968476,8.2,110 7th Ave W,CA,"110 7th Ave W, Vancouver BC V5Y 1L7",Vancouver,V5Y 1L7,BC,NaN,NaN
19,6993b3dbb0758927967592ea612a2b1e,4b82e001f964a5205aea30e3,49.282743,-123.129466,Nelson Park,0.995294,7.8,1030 Bute St,CA,"1030 Bute St (at Nelson St.), Vancouver BC V6E...",Vancouver,V6E 1Y7,BC,at Nelson St.,NaN


# Yelp API

In [23]:
# Sort the parks DataFrame by rating in descending order
filtered_yelp_df = merged_yelp_df.sort_values('rating', ascending=False)

# Print the sorted DataFrame
filtered_yelp_df.head(10)

,station_id,yelp_id,name,review_count,rating
19,32603a87cfca71d0f7dfa3513bad69d5,-fHqwIU4S-uD9O-ouzhE_A,Rainbow Park,3,5.0
17,32603a87cfca71d0f7dfa3513bad69d5,TMJ04QPJx-xf-UtpUk-AMQ,Granville Island Water Park,3,5.0
5,7a19c49f486d7c0c02b3685d7b240448,-65-OqxIBL4qdTN5PA-svw,Mount Pleasant Park,4,5.0
6,7a19c49f486d7c0c02b3685d7b240448,tI70Wf2bTa2eH2rURHYp5Q,Major Matthews Park,1,5.0
0,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
1,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
16,32603a87cfca71d0f7dfa3513bad69d5,pDB82PDofocYiHrH1lF88g,George Wainborn Park,9,4.5
13,32603a87cfca71d0f7dfa3513bad69d5,lqhQMVcOu6NDNIQKP7a0rg,David Lam Park,18,4.5
3,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5
7,7a19c49f486d7c0c02b3685d7b240448,yoCA7b04Q_f_HQX__EDchA,Heather Park,2,4.5


==================

In [31]:
#merged_fsq_df
#merged_yelp_df

fsq_yelp_df = pd.concat([merged_yelp_df, merged_fsq_df[['fsq_id', 'fsq_lat', 'fsq_long', 'name', 'popularity', 'rating']]], axis=1)


In [36]:
fsq_yelp_df.head()

,station_id,yelp_id,name,review_count,rating,fsq_id,fsq_lat,fsq_long,name,popularity,rating
0,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5,4be73b80910020a16435d514,49.264274,-123.108018,Jonathan Rogers Park,0.968476,8.2
1,7a19c49f486d7c0c02b3685d7b240448,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5,4aada430f964a5201f6120e3,49.266831,-123.124538,Charleson Park,0.992559,8.9
2,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5,4e700331d1649bf26fdfe4eb,49.256478,-123.105651,Mt. Pleasant Park,0.987758,8.4
3,32603a87cfca71d0f7dfa3513bad69d5,oCNy5AtXUagtQQQo0vqAtw,Charleson Park,20,4.5,4c0beac07e3fc928d200f682,49.270777,-123.109869,Hinge Park,0.985902,7.4
4,7a19c49f486d7c0c02b3685d7b240448,E43R3DuzaKD05c-Ngv-4Nw,Jonathan Rogers Park,11,4.5,4d486a1bf046a1cd8f7611f6,49.271004,-123.110781,Olympic Village Dog Run,0.875586,6.8


In [34]:
# create CSV file
fsq_yelp_df.to_csv('fsq_yelp_data.csv', index=False)